<a href="https://colab.research.google.com/github/Suhana-09-2005/NLP/blob/main/nlpassignment6_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd

df = pd.read_excel("/content/LDA-Data.xlsx")
print(df.head())
texts = df['News'].astype(str).tolist()

                             News
0   Virat scored century in match
1            BJP won in elections
2  Bumra took 5 wicket in a match
3  Congress form state government


In [15]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab') # Added to resolve LookupError

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = text.lower()                                # lowercase
    text = re.sub(r'[^a-z\s]', '', text)                # remove punctuation & numbers
    tokens = word_tokenize(text)                        # tokenize
    tokens = [t for t in tokens if t not in stop_words] # remove stopwords
    tokens = [lemmatizer.lemmatize(t) for t in tokens]  # lemmatize
    return tokens

processed = [clean_text(text) for text in texts]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [16]:
!pip install gensim
from gensim import corpora

dictionary = corpora.Dictionary(processed)
corpus = [dictionary.doc2bow(text) for text in processed]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 52.4 MB/s eta 0:00:00


In [17]:
import pandas as pd
import numpy as np

# Create BoW matrix
bow_matrix = []

for doc in corpus:
    row = np.zeros(len(dictionary), dtype=int)
    for word_id, count in doc:
        row[word_id] = count
    bow_matrix.append(row)

# Convert to DataFrame
bow_df = pd.DataFrame(
    bow_matrix,
    columns=[dictionary[id] for id in range(len(dictionary))]
)

print(bow_df.head())


   century  match  scored  virat  bjp  election  bumra  took  wicket  \
0        1      1       1      1    0         0      0     0       0   
1        0      0       0      0    1         1      0     0       0   
2        0      1       0      0    0         0      1     1       1   
3        0      0       0      0    0         0      0     0       0   

   congress  form  government  state  
0         0     0           0      0  
1         0     0           0      0  
2         0     0           0      0  
3         1     1           1      1  


In [18]:
import pandas as pd
import numpy as np

# Create Bag of Words matrix
bow_matrix = []

for doc in corpus:
    row = np.zeros(len(dictionary), dtype=int)
    for word_id, count in doc:
        row[word_id] = count
    bow_matrix.append(row)

# Convert to DataFrame
bow_df = pd.DataFrame(
    bow_matrix,
    columns=[dictionary[i] for i in range(len(dictionary))]
)

# Display first 5 rows
print(bow_df.head())


   century  match  scored  virat  bjp  election  bumra  took  wicket  \
0        1      1       1      1    0         0      0     0       0   
1        0      0       0      0    1         1      0     0       0   
2        0      1       0      0    0         0      1     1       1   
3        0      0       0      0    0         0      0     0       0   

   congress  form  government  state  
0         0     0           0      0  
1         0     0           0      0  
2         0     0           0      0  
3         1     1           1      1  


In [21]:
from gensim.models import LdaModel

# Train the LDA model
# You might need to adjust num_topics and passes based on your dataset
num_topics = 2 # Example: You can change this number
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, passes=10, random_state=100)

# Print all topics with top 10 words
for idx, topic in lda_model.print_topics(num_topics=-1, num_words=10):
    print(f"Topic {idx}:")
    print(topic)
    print()

Topic 0:
0.092*"state" + 0.092*"congress" + 0.092*"form" + 0.092*"government" + 0.091*"wicket" + 0.091*"took" + 0.091*"bumra" + 0.091*"election" + 0.091*"bjp" + 0.086*"match"

Topic 1:
0.149*"match" + 0.139*"century" + 0.139*"scored" + 0.139*"virat" + 0.049*"bjp" + 0.049*"election" + 0.049*"bumra" + 0.049*"took" + 0.048*"wicket" + 0.048*"government"



In [22]:
# Print all topics with top 10 words
for idx, topic in lda_model.print_topics(num_topics=-1, num_words=10):
    print(f"Topic {idx}:")
    print(topic)
    print()


Topic 0:
0.092*"state" + 0.092*"congress" + 0.092*"form" + 0.092*"government" + 0.091*"wicket" + 0.091*"took" + 0.091*"bumra" + 0.091*"election" + 0.091*"bjp" + 0.086*"match"

Topic 1:
0.149*"match" + 0.139*"century" + 0.139*"scored" + 0.139*"virat" + 0.049*"bjp" + 0.049*"election" + 0.049*"bumra" + 0.049*"took" + 0.048*"wicket" + 0.048*"government"



In [34]:
import pandas as pd

# Load the Excel file
file_path = "/content/LDA-Data (1).xlsx"
df = pd.read_excel(file_path, engine="openpyxl")

# --- Diagnosing the error: Print column names ---
print("DataFrame columns:", df.columns)

# Extract abstracts column, drop missing values, and ensure strings
# You will need to replace 'abstract' with the correct column name after inspecting the output above.
corpus = (
    df["News"]
    .dropna()
    .astype(str)
    .tolist()
)

# Sanity check
print(f"Number of documents: {len(corpus)}")
print("First document:\n", corpus[0])

DataFrame columns: Index(['News'], dtype='object')
Number of documents: 4
First document:
 Virat scored century in match


In [35]:
[
 "We propose a new deep learning approach...",
 "This paper discusses quantum computing...",
 ...
]


['We propose a new deep learning approach...',
 'This paper discusses quantum computing...',
 Ellipsis]

In [36]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)  # remove punctuation & numbers
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(word)
              for word in tokens if word not in stop_words]
    return " ".join(tokens)

processed_corpus = [preprocess(doc) for doc in corpus]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [37]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features=1000)
bow_matrix = vectorizer.fit_transform(processed_corpus)


In [38]:
bow_df = pd.DataFrame(
    bow_matrix.toarray(),
    columns=vectorizer.get_feature_names_out()
)

print(bow_df.head())


   bjp  bumra  century  congress  election  form  government  match  scored  \
0    0      0        1         0         0     0           0      1       1   
1    1      0        0         0         1     0           0      0       0   
2    0      1        0         0         0     0           0      1       0   
3    0      0        0         1         0     1           1      0       0   

   state  took  virat  wicket  
0      0     0      1       0  
1      0     0      0       0  
2      0     1      0       1  
3      1     0      0       0  


In [39]:
from sklearn.decomposition import NMF

num_topics = 5

nmf_model = NMF(
    n_components=num_topics,
    random_state=42
)

W = nmf_model.fit_transform(bow_matrix)
H = nmf_model.components_


In [40]:
feature_names = vectorizer.get_feature_names_out()

def display_topics(model, feature_names, num_words=10):
    for topic_idx, topic in enumerate(model.components_):
        print(f"\n🔹 Topic {topic_idx + 1}:")
        print(", ".join(
            [feature_names[i] for i in topic.argsort()[:-num_words - 1:-1]]
        ))

display_topics(nmf_model, feature_names)



🔹 Topic 1:
government, congress, state, form, wicket, scored, took, virat, match, election

🔹 Topic 2:
match, took, wicket, bumra, virat, scored, state, form, government, election

🔹 Topic 3:
virat, scored, century, match, wicket, state, took, form, government, election

🔹 Topic 4:
form, state, government, congress, wicket, took, bumra, scored, virat, election

🔹 Topic 5:
election, bjp, took, virat, wicket, scored, state, government, match, form


In [43]:
# Get dominant topic index for each document
df['dominant_topic'] = W.argmax(axis=1)

# Display abstract and its dominant topic
result = df[['News', 'dominant_topic']]

# Show first 5 rows
print(result.head())

                             News  dominant_topic
0   Virat scored century in match               2
1            BJP won in elections               4
2  Bumra took 5 wicket in a match               1
3  Congress form state government               0


In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=1000)
tfidf_matrix = vectorizer.fit_transform(processed_corpus)
